In [1]:
import os
import cv2
from pascal_voc_writer import Writer
import xml.etree.ElementTree as ET

In [2]:
train_folder_path = "Tensorflow\\workspace\\images\\train"
validation_folder_path = "Tensorflow\\workspace\\images\\validation"
test_folder_path = "Tensorflow\\workspace\\images\\test"

In [3]:
file_list_train = os.listdir(train_folder_path)
file_list_validation = os.listdir(validation_folder_path)
file_list_test = os.listdir(test_folder_path)

In [4]:
print("Total files in Training folder : ", len(file_list_train))
print("Total files in validation folder : ", len(file_list_validation))
print("Total files in Test folder : ", len(file_list_test))

Total files in Training folder :  3758
Total files in validation folder :  846
Total files in Test folder :  791


In [2]:
def xml_modifier(file_list,directory):
    
    cwd = os.getcwd()
    os.chdir(directory)
    count = 0    
    for file in file_list:
        file_name = file.split('.')[:-1] + ["xml"]
        file_ext = file.split('.')[-1]

        if (file_ext == 'xml'):
            mytree = ET.parse(file)

            myroot = mytree.getroot()

            # iterating through the name values.
            for names in myroot.iter('name'):
                # updates the name value
                names.text = str('NumberPlate')

            count+=1
            file_name_str = '.'.join([str(elem) for elem in file_name])
            mytree.write(file_name_str)
    print('Number of files Modified : ', count)
    os.chdir(cwd)


In [6]:
xml_modifier(file_list_train,train_folder_path)

Number of files Modified :  6891


In [7]:
xml_modifier(file_list_validation,validation_folder_path)

Number of files Modified :  1669


In [15]:
xml_modifier(file_list_test,test_folder_path)

Number of files Modified :  150


In [15]:
folder_path = "ilp\\State-wise_OLX\\validation_olx"
file_list = os.listdir(folder_path)
xml_modifier(file_list,folder_path)

Number of files Modified :  105


In [7]:
def xml_image_name_modifier(file_list,directory):
    
    cwd = os.getcwd()
    os.chdir(directory)
    count = 0    
    for file in file_list:
        file_name = file.split('.')[0] + ".jpg"
        file_ext = file.split('.')[-1]

        if (file_ext == 'xml'):
            mytree = ET.parse(file)

            myroot = mytree.getroot()

            # iterating through the name values.
            
            for names in myroot.iter('filename'):
                # updates the name value
                if names.text.split('.')[-1] == 'jpeg':
                    
                    names.text = str(file_name)

            count+=1
            mytree.write(file)
    print('Number of files Modified : ', count)
    os.chdir(cwd)

In [8]:
xml_image_name_modifier(file_list_train,train_folder_path)

Number of files Modified :  629


In [9]:
xml_image_name_modifier(file_list_validation,validation_folder_path)

Number of files Modified :  135


In [10]:
xml_image_name_modifier(file_list_test,test_folder_path)

Number of files Modified :  147


In [31]:
xpth = 'C:\\Users\\gs\\Documents\\python\\anpr\\Tensorflow\\workspace\\images\\validation\\00000003_7.xml'

In [13]:
source_file_path = os.path.join('Tensorflow','workspace','images','validation')
file_list = [file for file in os.listdir(source_file_path) 
             if os.path.isfile(os.path.join(source_file_path, file))]
    # file_list = os.listdir(source_file_path)
    # print('Number of images : ',len(file_list)//2)
improper_file_count = 0
file_count = 0
print(len(file_list))
for file in file_list:

    file_name = file.split('.')[0]
    file_ext = file.split('.')[-1]

    if (file_ext == 'xml'):
        
        xml_file_address = os.path.join(source_file_path, file)
        mytree = ET.parse(xml_file_address)

        myroot = mytree.getroot()
        file_count+=1

        # iterating through the bndbox values.

        for dimension in myroot.iter('size'):
            height = int(dimension.find('height').text)
            width = int(dimension.find('width').text)

        if (height <= 0) | (width <= 0):
            improper_file_count+=1
            print('width/height <= 0 : ', file)

        if myroot.findall('object') == []:
            continue

        else:
            for box in myroot.iter('bndbox'):
                xmin = int(box.find('xmin').text)
                ymin = int(box.find('ymin').text)
                xmax = int(box.find('xmax').text)
                ymax = int(box.find('ymax').text)

                xmn = xmin / width
                xmx = xmax / width

                ymn = ymin / height
                ymx = ymax / height
                
                box_width = xmax - xmin
                box_height = ymax - ymin
                
                area = box_width * box_height
                if area == 0 or area < 0:
                      print(f"Bbx with area = {area} not allowed")
                    
                if (xmin>=xmax) | (ymin>=ymax):
                    print('min > max : ', file)

                if (xmin<0) | (ymin<0) | (xmax<0) | (ymax<0):
                    improper_file_count+=1
                    print('binding box < 0 : ', file)

                if (xmn < 0.0) | (ymn < 0.0) | (xmx < 0.0) | (ymx < 0.0):
                    improper_file_count+=1
                    print('ratio of bbox/size < 0 : ', file)
                if (xmn > 1) | (ymn > 1) | (xmx > 1) | (ymx > 1):
                    improper_file_count+=1
                    print('ratio of bbox/size > 0 : ', file)
                
print(file_count)

846
423


In [12]:
file_count

1879

In [34]:
source_file_path = os.path.join('Tensorflow','workspace','images','train')
source_file_path

'Tensorflow\\workspace\\images\\train'

## Finding missing jpg of xml file

In [31]:
def missing_jpg(xml_file_path,jpg_file_path):
    
    file_list = [file for file in os.listdir(xml_file_path) 
             if os.path.isfile(os.path.join(xml_file_path, file))]
    # file_list = os.listdir(xml_file_path)
    print('Number of files : ',len(file_list))
    blank_file_count = 0
    file_count = 0
    for file in file_list:

        file_name = file.split('.')[0]
        file_ext = file.split('.')[1]

        if (file_ext == 'jpg'):
            
            # print((os.path.join(xml_file_path,file_name,f'.jpg'))
            path = os.path.join(xml_file_path, f'{file_name}.xml')
            if not os.path.exists(path):
                
                print(file)
            # img = cv2.imread(os.path.join(jpg_file_path,f'{file_name}.jpg'))
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [34]:
xml_file_path = os.path.join('Tensorflow','workspace','images','train')
jpg_file_path = os.path.join('Tensorflow','workspace','images','train')
missing_jpg(xml_file_path,jpg_file_path)

Number of files :  3752


## creating new xml files

In [26]:
def creating_new_xml_files(source_file_path,destination_file_path):
    
    file_list = [file for file in os.listdir(destination_file_path) 
             if os.path.isfile(os.path.join(destination_file_path, file))]
    # file_list = os.listdir(source_file_path)
    # print('Number of images : ',len(file_list)//2)
    file_count = 0
    print('Total number of jpg files : ',len(file_list))
    for file in file_list:

        file_name = file.split('.')[0]
        file_ext = file.split('.')[-1]

        if (file_ext == 'jpg'):
            
            xml_file_address = os.path.join(source_file_path,f'{file_name}.xml')
            jpg_file_address = os.path.join(destination_file_path,f'{file_name}.jpg')
            # print(jpg_file_address)
            
            im = cv2.imread(jpg_file_address)
            # print(im)
            height = (im.shape[0])
            width = (im.shape[1])
            
            mytree = ET.parse(xml_file_address)

            myroot = mytree.getroot()
            
            # iterating through the size values.
            # for dimension in myroot.iter('size'):
            #     width = int(dimension.find('width').text)
            #     height = int(dimension.find('height').text)
            #     depth = int(dimension.find('depth').text)
                

            # iterating through the bndbox values.
            bbox = []
            for box in myroot.iter('bndbox'):
                bbox_temp = []
                bbox_temp.append(int(box.find('xmin').text))
                bbox_temp.append(int(box.find('ymin').text))
                bbox_temp.append(int(box.find('xmax').text))
                bbox_temp.append(int(box.find('ymax').text))
                bbox_temp.append('NumberPlate')
                bbox.append(bbox_temp)
                
            # create pascal voc writer (image_path, width, height)
            writer = Writer(os.path.join(destination_file_path, f'{file_name}.jpg'), 
                            width, height)
            # add objects (class, xmin, ymin, xmax, ymax)
            for info in (bbox):
                writer.addObject(info[4], round(info[0]), round(info[1]), round(info[2]), round(info[3]))
            # write to file
            writer.save(os.path.join(destination_file_path, f'{file_name}.xml'))
            file_count+=1

           
    print('Total number of xml files created : ', file_count)

In [27]:
xml_file_path = os.path.join('Tensorflow','workspace','images','train','train_xml')
jpg_file_path = os.path.join('Tensorflow','workspace','images','train','train_img')
creating_new_xml_files(xml_file_path,jpg_file_path)

Total number of jpg files :  1877
Total number of xml files created :  1875


In [6]:
os.getcwd()

'C:\\Users\\gs\\Documents\\python\\anpr'